#### 如果要用關鍵字爬取facebook的內容，要先
#### 登入（希望能自動化）
#### 但上面的結果告訴我們，由一般網頁登入後也不見得能取得包括內容的原始碼
#### 我們可以改由簡易版facebook進入：mbasic.facebook.com

In [1]:
# 請用上週學的方法，讓程式自動登入mbasic.facebook.com

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

In [2]:
username = input('請輸入你的帳號：')
password = input('請輸入你的密碼：')
kword = input('請輸入你想找的關鍵詞：')

請輸入你的帳號：np273838@gmail.com
請輸入你的密碼：838372pn
請輸入你想找的關鍵詞：台師大


In [3]:
path = r'C:\\Users\\user\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe' #桌機user，筆電改sengh
service=Service(executable_path=path)
service

In [4]:
chrome=webdriver.Chrome(service=service)
chrome.get('https://mbasic.facebook.com')
time.sleep(5)

In [5]:
un_xpath = '/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[1]/input'
# 讓程式根據xpath找到輸入框
un_element = chrome.find_element(By.XPATH, un_xpath)
un_element

<selenium.webdriver.remote.webelement.WebElement (session="9a99a0766d7fc0704f13a1559e1368b7", element="AEAE7C9E20C06F143D785A6D447ABE09_element_5")>

In [6]:
# 讓程式在輸入框自動輸入你的帳號

un_element.send_keys(username)

In [7]:
# 讓程式定位密碼輸入框

pw_xpath='/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[2]/section/input'

pw_element = chrome.find_element(By.XPATH, pw_xpath)
pw_element

<selenium.webdriver.remote.webelement.WebElement (session="9a99a0766d7fc0704f13a1559e1368b7", element="AEAE7C9E20C06F143D785A6D447ABE09_element_8")>

In [8]:
# 讓程式在輸入框自動輸入你的密碼

pw_element.send_keys(password)

In [9]:
# 按下登入按鈕
input_xpath = '/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[3]/input'
input_button = chrome.find_element(By.XPATH, input_xpath)
input_button.click()

In [10]:
# 有時會出現one tap登入的頁面；無妨，要瀏覽器再指向網址即可

chrome.get('https://mbasic.facebook.com')

In [11]:
# 搜尋框定位

kw_xpath = '/html/body/div/div/div[1]/div/form/table/tbody/tr/td[2]/input'
kw_element = chrome.find_element(By.XPATH, kw_xpath)
kw_element

<selenium.webdriver.remote.webelement.WebElement (session="9a99a0766d7fc0704f13a1559e1368b7", element="FDA4FAA2A6A8DC394F5E85918FD8CFCC_element_38")>

In [12]:
# 輸入關鍵詞

kw_element.send_keys(kword)

In [13]:
# 定位搜尋按鈕，按下按鈕

kwb_xpath = '/html/body/div/div/div[1]/div/form/table/tbody/tr/td[3]/input'
kw_button = chrome.find_element(By.XPATH, kwb_xpath)
kw_button.click()

In [14]:
p1 = chrome.page_source

In [15]:
from bs4 import BeautifulSoup

p1_soup = BeautifulSoup(p1, 'lxml')
p1_soup

<html lang="zh-Hant" xmlns="http://www.w3.org/1999/xhtml"><head><title>「台師大」的搜尋結果</title><meta content="origin-when-crossorigin" id="meta_referrer" name="referrer"/><style nonce="" type="text/css">/*<![CDATA[*/.bv{margin:4px 10px;}.bv.bw{padding:4px;}.b .bv.du{margin:10px 3px;}.b .ds{border:0;margin:0;}.b .dv{border:1px solid #bec3c9;margin:0 6px;}.b .bq .bv .ca{padding:5px;}.b .bq .bv .cs{background:#fff;border-top:1px solid #dadde1;color:#4267b2;display:block;padding:5px;text-align:center;}.by{border-bottom:1px solid #dadde1;}@keyframes rotateSearchLiveFeedSpinner{0%{transform:rotate(0deg);}100%{transform:rotate(360deg);}}.dw{font-weight:normal;}.dw a{font-weight:bold;}.bk{background:#fff;border-bottom:1px solid #dadde1;}.bm{display:inline-block;}.bl{background:#fff;position:relative;}.bn,.bp{border-bottom:2px solid transparent;display:inline-block;padding:5px 16px 2px;}.b a,.b a:visited{color:#3b5998;text-decoration:none;}.b .ex,.b .ex:visited{color:gray;}.b .bg,.b .bg:visited{color

# 有找到作者的名字所在div、td的標籤都會變，用h3，缺點，會抓到Groups、Events、...commented on this
# 用h3裡有strong來抓

authors = []

p1_authors = p1_soup.find_all('h3')

for aut in p1_authors:
    if aut.find('strong')!=None:
        authors.append(aut.text.strip())

authors

#### 只要用selenium每翻一次頁，就找一次作者名，就可以累積作者名的串列
#### 翻頁連結在 div id="see_more_pager" 裡的 a tag 裡的連結

next_p = p1_soup.find('div', id="see_more_pager").find('a').get('href')

chrome.get(next_p)

In [16]:
# 寫成迴圈
# 第一頁找到後，翻99次
# 所以authors保留之前第一頁找到的elements
# 把上面程式碼貼下來
# 中間停3秒
# 可能沒那麼多頁，會找不到翻頁而報錯，所以要try except
authors=[]
for i in range(0, 99):
    try:
        p = chrome.page_source
        soup = BeautifulSoup(p, 'lxml')
        p_authors = soup.find_all('h3')
        for aut in p_authors:
            if aut.find('strong')!=None:
                authors.append(aut.text.strip()[0:12]) #拿掉作者發文社團及心情
        time.sleep(3)
        chrome.get(soup.find('div', id="see_more_pager").find('a').get('href'))
    except:
        break

authors

['田克強 > \u200e臺師大最溫',
 '席耶娜──在 Bar n',
 '告白師大  NTNU C',
 '台師大男乙籃',
 '臺師大學生會 NTNU ',
 '莊佳穎',
 '台大台語文社-ntutg',
 '國立臺灣師範大學 Nat',
 '金佳瑜 > \u200e師大台文1',
 '國立臺灣師範大學 Nat',
 '師範範 > \u200e臺師大最溫',
 'Li Bichhin',
 '告白師大  NTNU C',
 '陳歆絜 > \u200e臺師大最溫',
 '國立臺灣師範大學 Nat',
 'Tsiȯh Bȯk-bî',
 '台灣也有一個騰訊',
 '告白師大  NTNU C',
 '國立臺灣師範大學 Nat',
 '王向銘 > \u200e臺師大最溫',
 'Yingying Che',
 '台灣也有一個騰訊',
 'Li Bichhin',
 '打開台大｜Open Ho',
 '國立臺灣師範大學 Nat',
 '國立臺灣師範大學 Nat',
 '金曲 GMA',
 'TVBS 新聞',
 'Yachi Chiang',
 '張勝煇',
 '永年高級中學──在台大圖',
 '國立臺灣師範大學 Nat',
 '市立南化國中',
 '曹春婷──和林志及林祐民',
 '謝翰哲',
 '靠北排球2.0',
 '王盈勛',
 '國立臺灣師範大學國際版畫',
 '國立臺灣師範大學 Nat',
 '陳傳仁',
 '宋美嫺',
 '蘇楠──在輔仁大學。',
 '馬世芳',
 '國立臺灣師範大學 Nat',
 'Carlo Lin',
 '何傑恩',
 '黃慧真──和林佳蓉及其他',
 '國立臺灣師範大學 Nat',
 '台大台語文社-ntutg',
 '台師大科教所',
 '國立臺灣師範大學 Nat',
 '馬世芳',
 '林英明',
 '罕見疾病基金會',
 '許淑貞──和賈生玲及吳慧',
 '許祐子',
 '郭旭展──在國立臺灣師範',
 '宋美嫺',
 '張絲媛──在國立臺灣師範',
 '葉欣誠',
 '國立臺灣師範大學 Nat',
 '財團法人李江却台語文教基',
 '台師大科教所',
 '林彩媚',
 'Horatio Han─',
 '林政宏',
 '林昀柔──在國立臺灣師範',
 '台師大科教所',
 'Catherine Le',
 '林怡慧

In [17]:
# 計算作者發文數

aut_count={}

for aut in authors:
    if aut not in aut_count:
        aut_count[aut]=1
    else:
        aut_count[aut]+=1
        
aut_count

{'田克強 > \u200e臺師大最溫': 1,
 '席耶娜──在 Bar n': 1,
 '告白師大  NTNU C': 4,
 '台師大男乙籃': 1,
 '臺師大學生會 NTNU ': 1,
 '莊佳穎': 1,
 '台大台語文社-ntutg': 2,
 '國立臺灣師範大學 Nat': 13,
 '金佳瑜 > \u200e師大台文1': 1,
 '師範範 > \u200e臺師大最溫': 1,
 'Li Bichhin': 2,
 '陳歆絜 > \u200e臺師大最溫': 1,
 'Tsiȯh Bȯk-bî': 1,
 '台灣也有一個騰訊': 2,
 '王向銘 > \u200e臺師大最溫': 1,
 'Yingying Che': 1,
 '打開台大｜Open Ho': 1,
 '金曲 GMA': 1,
 'TVBS 新聞': 1,
 'Yachi Chiang': 1,
 '張勝煇': 1,
 '永年高級中學──在台大圖': 1,
 '市立南化國中': 1,
 '曹春婷──和林志及林祐民': 1,
 '謝翰哲': 1,
 '靠北排球2.0': 1,
 '王盈勛': 1,
 '國立臺灣師範大學國際版畫': 1,
 '陳傳仁': 1,
 '宋美嫺': 2,
 '蘇楠──在輔仁大學。': 1,
 '馬世芳': 3,
 'Carlo Lin': 1,
 '何傑恩': 1,
 '黃慧真──和林佳蓉及其他': 1,
 '台師大科教所': 3,
 '林英明': 1,
 '罕見疾病基金會': 1,
 '許淑貞──和賈生玲及吳慧': 1,
 '許祐子': 1,
 '郭旭展──在國立臺灣師範': 1,
 '張絲媛──在國立臺灣師範': 1,
 '葉欣誠': 1,
 '財團法人李江却台語文教基': 1,
 '林彩媚': 1,
 'Horatio Han─': 1,
 '林政宏': 1,
 '林昀柔──在國立臺灣師範': 1,
 'Catherine Le': 1,
 '林怡慧': 1,
 'HSNUmusic 師大': 1,
 '2024臺師大電機營': 1,
 'IOH Innovati': 1,
 '自由時報': 1,
 '敢動傳媒': 1,
 '周秀甄': 1,
 '中華民國體育學會──覺得': 1,
 '央廣臺灣台語迷眾頁': 1,
 '台語語法冷知識': 1,
 '劉定綱

In [18]:
max_value = max(aut_count.values())
for key, value in aut_count.items():
    if value == max_value:
        print(f'發文數最多的是{key}，共計{aut_count[key]}篇。')


發文數最多的是國立臺灣師範大學 Nat，共計13篇。


In [19]:
# 修改程式，讓結果
# 列出總篇數
# 前三大發文者及這三個人各自的發文篇數

sorted_dict = dict(sorted(aut_count.items(), key=lambda item: item[1], reverse=True))
biggest_three = list(sorted_dict.keys())[:3]

print(f'總篇數為{sum(aut_count.values())}篇，前三大發文者依序為:{biggest_three[0]}共{sorted_dict[biggest_three[0]]}篇、{biggest_three[1]}共{sorted_dict[biggest_three[1]]}篇、{biggest_three[2]}共{sorted_dict[biggest_three[2]]}篇。')

總篇數為88篇，前三大發文者依序為:國立臺灣師範大學 Nat共13篇、告白師大  NTNU C共4篇、馬世芳共3篇。


In [ ]:
# 引入tkinter套件，啟動視窗並指定大小及標頭
import tkinter as tk

win=tk.Tk()
win.geometry('300x200')
win.title('面冊主題發言追蹤')

# 加入標籤
label1=tk.Label(win,text='口座', width=10, height=2, bg='black', fg='white')
label1.pack() #指定等下用pack方式排版

win.mainloop() # 會一直執行等待回應，一直到關閉視窗為止

In [ ]:
# 引入tkinter套件，啟動視窗並指定大小及標頭
import tkinter as tk

win=tk.Tk()
win.geometry('300x200')
win.title('面冊主題發言追蹤')

# 加入標籤
label1=tk.Label(win,text='口座')
label1.pack(side='top', anchor='n') #指定等下用pack方式排版

label2=tk.Label(win,text='密碼')
label2.pack(side='bottom', anchor='e')

label3=tk.Label(win,text='關鍵字')
label3.pack(side='left', anchor='w')

win.mainloop() # 會一直執行等待回應，一直到關閉視窗為止


In [ ]:
# 引入tkinter套件，啟動視窗並指定大小及標頭
import tkinter as tk

win=tk.Tk()
win.geometry('300x200')
win.title('面冊主題發言追蹤')

# 加入標籤
label1=tk.Label(win,text='口座')

label2=tk.Label(win,text='密碼')

label3=tk.Label(win,text='關鍵字')

# 顯示搜尋結果的區域也是個標籤
label4 = tk.Label(win,text='結果',textvariable=labelVar, fg='blue')


# 加入輸入框
entry1=tk.Entry(win)  #一樣可以加入字型等變數

entry2=tk.Entry(win)

entry3=tk.Entry(win)

# 加入按鈕
button = tk.Button(win, text='開始搜揣', command=btn_click)

# .pack的順序會決定各元件在視窗上出現的位置順序
label1.pack()
entry1.pack()
label2.pack()
entry2.pack()
label3.pack()
entry3.pack()
label4.pack()
button.pack()

win.mainloop() # 會一直執行等待回應，一直到關閉視窗為止


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
import tkinter as tk
from bs4 import BeautifulSoup

def btn_click():
    path = r'C:\Users\sengh\chromedriver-win64\chromedriver.exe' #桌機user，筆電改sengh
    service=Service(executable_path=path)
    chrome=webdriver.Chrome(service=service)
    chrome.get('https://mbasic.facebook.com')
    time.sleep(5)
    un_xpath = '/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[1]/input'
    un_element = chrome.find_element(By.XPATH, un_xpath)
    # 由視窗輸入文字抓取帳號
    username=entry1.get()
    un_element.send_keys(username)
    pw_xpath='/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[2]/section/input'
    pw_element = chrome.find_element(By.XPATH, pw_xpath)
    # 由視窗輸入文字抓取密碼
    password=entry2.get()
    pw_element.send_keys(password)
    input_xpath = '/html/body/div/div/div[2]/div/table/tbody/tr/td/div[2]/div/div[2]/form/ul/li[3]/input'
    input_button = chrome.find_element(By.XPATH, input_xpath)
    input_button.click()
    chrome.get('https://mbasic.facebook.com')
    kw_xpath = '/html/body/div/div/div[1]/div/form/table/tbody/tr/td[2]/input'
    kw_element = chrome.find_element(By.XPATH, kw_xpath)
    # 由視窗輸入文字抓取關鍵字
    kword=entry3.get()
    kw_element.send_keys(kword)
    kwb_xpath = '/html/body/div/div/div[1]/div/form/table/tbody/tr/td[3]/input'
    kw_button = chrome.find_element(By.XPATH, kwb_xpath)
    kw_button.click()
    p1 = chrome.page_source
    p1_soup = BeautifulSoup(p1, 'lxml')
    authors=[]
    for i in range(0, 99):
        try:
            p = chrome.page_source
            soup = BeautifulSoup(p, 'lxml')
            p_authors = soup.find_all('h3')
            for aut in p_authors:
                if aut.find('strong')!=None:
                    authors.append(aut.text.strip()[0:12]) #拿掉作者發文社團及心情
            time.sleep(3)
            chrome.get(soup.find('div', id="see_more_pager").find('a').get('href'))
        except:
            break
    aut_count={}
    for aut in authors:
        if aut not in aut_count:
            aut_count[aut]=1
        else:
            aut_count[aut]+=1
    sorted_dict = dict(sorted(aut_count.items(), key=lambda item: item[1], reverse=True))
    biggest_three = list(sorted_dict.keys())[:3]
    print(f'總篇數為{sum(aut_count.values())}篇，前三大發文者依序為:{biggest_three[0]}共{sorted_dict[biggest_three[0]]}篇、{biggest_three[1]}共{sorted_dict[biggest_three[1]]}篇、{biggest_three[2]}共{sorted_dict[biggest_three[2]]}篇。')